In [1]:
import pandas as pd

processed_data_path = "C:/Users/anush/OneDrive/Documents/Sem3/Explorative Information Visualization/Project/Data/CleanedData"

pizza_ingredients = ["wheat", "tomato", "onion", "peppers", "vegetables", "fruits"]
country_list = ["India", "USA", "Egypt"]
year_min = 1979
year_max = 2019 #new max year is 2019 (not 2023)

### Importing already preprocessed data

In [2]:
#Food Loss Percentage
foodLoss = pd.read_csv(processed_data_path + "/country-commodity-yearwise-timeseries-data-interpolatedData.csv")

In [3]:
foodLoss.head(10)

,commodity,country,1979,1980,1981,1982,1983,1984,1985,1986,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,onion,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,20.00,20.00,...,20.00,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
1,peppers,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,20.00,20.00,...,20.00,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
2,tomato,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,43.00,43.00,...,43.00,43.0000,43.000,43.0000,29.00,15.000,15.00,15.00,15.00,15.00
3,wheat,Egypt,20.00,20.00,20.00,20.00,20.00,20.00,7.20,7.20,...,14.00,14.0000,14.000,14.0000,14.00,14.000,14.00,14.00,14.00,14.00
4,onion,India,30.00,30.00,30.00,30.00,30.00,30.00,30.00,30.00,...,27.50,24.7225,21.945,19.1675,16.39,26.815,37.24,37.24,37.24,37.24
5,peppers,India,10.19,10.19,10.19,10.19,10.19,10.19,10.19,10.19,...,41.67,34.5000,27.330,20.1600,12.99,10.550,8.11,8.11,8.11,8.11
6,tomato,India,13.00,13.00,13.00,13.00,13.00,13.00,13.00,13.00,...,78.75,56.6400,34.540,12.4400,24.88,35.000,28.04,28.04,28.04,28.04
7,wheat,India,4.95,4.95,4.95,4.95,4.95,4.95,4.95,4.95,...,10.41,10.4100,12.930,34.4200,9.85,15.300,24.19,24.19,24.19,24.19
8,onion,USA,6.00,6.00,6.00,6.00,6.00,6.00,6.00,6.00,...,6.00,6.0000,6.000,6.0000,6.00,6.000,6.00,6.00,6.00,6.00
9,peppers,USA,8.00,8.00,8.00,8.00,8.00,8.00,8.00,8.00,...,8.00,8.0000,8.000,8.0000,8.00,8.000,8.00,8.00,8.00,8.00


In [4]:
#emissions_kg_co2eq - Greenhouse gas emissions (in kg of CO2 eq) to produce 1kg of crop
#water_use_L - L of water used in producing 1kg of crop
#land_use_sq_m - Land used in producing 1kg of crop
envMetrics = pd.read_csv(processed_data_path + "/EnvironmentalMetrics_ingredients.csv")

In [5]:
envMetrics.head()

,Entity,emissions_kg_co2eq,water_use_L,land_use_sq_m
0,onion,0.50,53.86,0.39
1,tomato,2.09,291.67,0.80
2,wheat,1.57,648.00,3.85
3,peppers,0.53,362.93,1.24


In [6]:
#Value produced is in tonnes
foodProduced = pd.read_csv(processed_data_path + "/countrywise-food-through-years.csv")
mask=[]
for idx, row in foodProduced.iterrows():
    flag = 0
    for c in country_list:
        if c in row["country"]:
            mask.append(True)
            flag = 1
            break
    if flag == 0:
        mask.append(False)
        
foodProduced = foodProduced[mask]

In [7]:
set(foodProduced["country"])

{'Egypt', 'India', 'USA'}

In [8]:
foodProduced.head()

,country,year,peppers,onion,tomato,wheat
0,India,1979,7.2973,10.232100,8.666699,1.5683
1,India,1980,7.3077,9.961400,9.375000,1.4356
2,India,1981,7.3333,10.561999,9.473700,1.6299
3,India,1982,7.7000,10.330400,9.722199,1.6913
4,India,1983,7.7073,9.981900,10.000000,1.8158


### Creating separate tables for easier plotting

In [9]:
#seperate tables for CO2eq, waterUse and landUse

#totalFood_tonnes = ... #This will be used for the size of the plate
#totalWaterUsed_perCountry = ...
#totalCO2emission_perCountry = ...
#totalLandUsed_perCountry = ...

#### Total Food produced in tonnes in each country each year

In [10]:
totalFood_tonnes_column = []
for idx, row in foodProduced.iterrows():
    foodsum = row["peppers"] + row["onion"] + row["tomato"] + row["wheat"]
    totalFood_tonnes_column.append(foodsum)
    
totalFood_tonnes = pd.DataFrame({"country": foodProduced["country"].to_list(),
                    "year": foodProduced["year"].to_list(), 
                    "totalFoodProduced_t": totalFood_tonnes_column}).round(2)

In [11]:
totalFood_tonnes.head()

,country,year,totalFoodProduced_t
0,India,1979,27.76
1,India,1980,28.08
2,India,1981,29.00
3,India,1982,29.44
4,India,1983,29.50


#### Total water used per country per crops to produce the food

In [12]:
# 1 tonne is 1000kg
waterUseLookup = dict(zip(envMetrics.Entity, envMetrics.water_use_L))
totalWater_column = []

for idx, row in foodProduced.iterrows():
    waterCalc = row["peppers"]*1000*waterUseLookup["peppers"] + row["onion"]*1000*waterUseLookup["onion"] + row["tomato"]*1000*waterUseLookup["tomato"] + row["wheat"]*1000*waterUseLookup["wheat"]
    waterCalc = waterCalc/100000
    totalWater_column.append(waterCalc)
    
totalWaterUsed_perCountry = pd.DataFrame({"country": foodProduced["country"].to_list(),
                            "year": foodProduced["year"].to_list(), 
                            "totalWaterUsed_kL_by100": totalWater_column}).round(2)

In [13]:
totalWaterUsed_perCountry

,country,year,totalWaterUsed_kL_by100
0,India,1979,67.44
1,India,1980,68.53
2,India,1981,70.50
3,India,1982,72.83
4,India,1983,74.28
...,...,...,...
127,Egypt,2018,243.89
128,Egypt,2019,244.15
129,Egypt,2020,245.62
130,Egypt,2021,251.88


#### Total emissions produced per country per crops to produce the food

In [14]:
emissionLookup = dict(zip(envMetrics.Entity, envMetrics.emissions_kg_co2eq))
totalEmissions_column = []

for idx, row in foodProduced.iterrows():
    emissionsCalc = row["peppers"]*1000*emissionLookup["peppers"] + row["onion"]*1000*emissionLookup["onion"] + row["tomato"]*1000*emissionLookup["tomato"] + row["wheat"]*1000*emissionLookup["wheat"]
    emissionsCalc = emissionsCalc/1000
    totalEmissions_column.append(emissionsCalc)
    
totalCO2emission_perCountry = pd.DataFrame({"country": foodProduced["country"].to_list(),
                              "year": foodProduced["year"].to_list(), 
                              "totalEmissions_t": totalEmissions_column}).round(2)

In [15]:
totalCO2emission_perCountry

,country,year,totalEmissions_t
0,India,1979,29.56
1,India,1980,30.70
2,India,1981,31.53
3,India,1982,32.22
4,India,1983,32.83
...,...,...,...
127,Egypt,2018,119.13
128,Egypt,2019,119.72
129,Egypt,2020,122.48
130,Egypt,2021,126.84


#### Total Food Lost

In [16]:
foodLoss.head()

,commodity,country,1979,1980,1981,1982,1983,1984,1985,1986,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,onion,Egypt,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
1,peppers,Egypt,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,...,20.0,20.0000,20.000,20.0000,20.00,23.000,23.00,23.00,23.00,23.00
2,tomato,Egypt,20.0,20.0,20.0,20.0,20.0,20.0,43.0,43.0,...,43.0,43.0000,43.000,43.0000,29.00,15.000,15.00,15.00,15.00,15.00
3,wheat,Egypt,20.0,20.0,20.0,20.0,20.0,20.0,7.2,7.2,...,14.0,14.0000,14.000,14.0000,14.00,14.000,14.00,14.00,14.00,14.00
4,onion,India,30.0,30.0,30.0,30.0,30.0,30.0,30.0,30.0,...,27.5,24.7225,21.945,19.1675,16.39,26.815,37.24,37.24,37.24,37.24


In [17]:
foodLoss.columns

Index(['commodity', 'country', '1979', '1980', '1981', '1982', '1983', '1984',
       '1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993',
       '1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

In [18]:
totalFoodLost_column = []
for idx, row in foodProduced.iterrows():
    yr = row["year"]
    c = row["country"]
    if yr > year_max:
        totalFoodLost_column.append(pd.NA)
        continue
    #get the rows that have the same country:
    foodLoss_filter = foodLoss[foodLoss["country"] == c]
    
    lossCalc = 0.0
    for i, commodityRow in foodLoss_filter.iterrows():
        lossCalc += row[commodityRow["commodity"]] * commodityRow[str(yr)] * 0.01
    
    totalFoodLost_column.append(lossCalc)
    
totalFoodLost_perCountry = pd.DataFrame({"country": foodProduced["country"].to_list(),
                              "year": foodProduced["year"].to_list(), 
                              "totalFoodLost_t": totalFoodLost_column})

In [19]:
totalFoodLost_perCountry

,country,year,totalFoodLost_t
0,India,1979,5.017527
1,India,1980,5.022887
2,India,1981,5.228124
3,India,1982,5.231355
4,India,1983,5.169826
...,...,...,...
127,Egypt,2018,19.406084
128,Egypt,2019,19.381085
129,Egypt,2020,<NA>
130,Egypt,2021,<NA>


#### Combining all data into a single dataframe

In [20]:
fullData = pd.DataFrame({"country": foodProduced["country"].to_list(),
                         "year": foodProduced["year"].to_list(),
                         "peppers": foodProduced["peppers"].to_list(),
                         "tomato": foodProduced["tomato"].to_list(),
                         "onion": foodProduced["onion"].to_list(),
                         "wheat": foodProduced["wheat"].to_list(),
                         "totalFoodProduced_t": totalFood_tonnes_column,
                         "totalWaterUsed_kL_by100": totalWater_column,
                         "totalEmissions_t": totalEmissions_column,
                         "totalFoodLost_t": totalFoodLost_column})

In [21]:
fullData

,country,year,peppers,tomato,onion,wheat,totalFoodProduced_t,totalWaterUsed_kL_by100,totalEmissions_t,totalFoodLost_t
0,India,1979,7.297300,8.666699,10.232100,1.5683,27.764399,67.435845,29.559251,5.017527
1,India,1980,7.307700,9.375000,9.961400,1.4356,28.079700,68.533794,30.701423,5.022887
2,India,1981,7.333300,9.473700,10.561999,1.6299,28.998899,70.497130,31.526624,5.228124
3,India,1982,7.700000,9.722199,10.330400,1.6913,29.443898,72.825924,32.220937,5.231355
4,India,1983,7.707300,10.000000,9.981900,1.8158,29.505000,74.281738,32.826625,5.169826
...,...,...,...,...,...,...,...,...,...,...
127,Egypt,2018,19.492699,38.773900,35.763000,6.2942,100.323799,243.885054,119.131975,19.406084
128,Egypt,2019,19.000000,39.000000,35.874100,6.5003,100.374400,244.151734,119.722521,19.381085
129,Egypt,2020,18.353500,40.671600,35.514100,6.3663,100.905500,245.618729,122.483139,<NA>
130,Egypt,2021,17.687700,42.607700,35.318700,6.8505,102.464600,251.881737,126.839208,<NA>


In [22]:
fullData = fullData.dropna()
fullData

,country,year,peppers,tomato,onion,wheat,totalFoodProduced_t,totalWaterUsed_kL_by100,totalEmissions_t,totalFoodLost_t
0,India,1979,7.297300,8.666699,10.232100,1.5683,27.764399,67.435845,29.559251,5.017527
1,India,1980,7.307700,9.375000,9.961400,1.4356,28.079700,68.533794,30.701423,5.022887
2,India,1981,7.333300,9.473700,10.561999,1.6299,28.998899,70.497130,31.526624,5.228124
3,India,1982,7.700000,9.722199,10.330400,1.6913,29.443898,72.825924,32.220937,5.231355
4,India,1983,7.707300,10.000000,9.981900,1.8158,29.505000,74.281738,32.826625,5.169826
...,...,...,...,...,...,...,...,...,...,...
124,Egypt,2015,15.043900,39.307600,36.723698,6.5919,97.667098,231.742199,118.837283,18.725554
125,Egypt,2016,15.037299,39.577400,35.413900,6.6311,96.659699,232.053724,118.804311,18.46874
126,Egypt,2017,17.529200,40.485900,36.414800,6.8597,101.289600,245.767815,122.883136,19.440363
127,Egypt,2018,19.492699,38.773900,35.763000,6.2942,100.323799,243.885054,119.131975,19.406084


#### Percentage of Food Lost (for Piechart)

In [25]:
percentageFoodLost_column = []
for idx, row in fullData.iterrows():
    percentage = row["totalFoodLost_t"]*100 / row["totalFoodProduced_t"]
    print(percentage)
    percentageFoodLost_column.append(percentage)
    

percentLossDF = pd.DataFrame({"percentageLoss": percentageFoodLost_column})
fullData = pd.concat([fullData, percentLossDF], axis=1)

18.071799753346
17.88796485016528
18.02869843905037
17.7671957496294
17.521864063703074
17.993726942988093
17.652563840463515
17.2473460135961
17.521573768924316
17.03751185632048
16.7670359534103
16.72574345678395
16.94017711064535
16.67099160955918
16.686394074035462
17.624673130272406
18.376809921891304
19.357970417981704
20.34145130210512
21.12406334956437
22.653862328153927
21.746789259259533
21.257705532283094
22.0813399835972
15.261865735873176
14.809904979779319
16.189181995876986
21.188618355089453
23.62969435386333
26.09028542394497
16.262138979511096
51.31589893027099
38.63689762687081
27.671305344246868
17.45043821450271
19.063636862897216
26.94047065578923
27.37021358432648
27.623450757785232
27.77918262132048
27.881301203702133
10.538772779980532
10.595331706463377
10.38925128147939
10.511450939457044
10.452255639510293
10.532932305995834
10.412539513738094
10.468954471321496
10.530693095231864
10.368777437654455
10.56417225456358
10.49114779095373
10.566519079372457
10.5

In [28]:
fullData.head()

,country,year,peppers,tomato,onion,wheat,totalFoodProduced_t,totalWaterUsed_kL_by100,totalEmissions_t,totalFoodLost_t,percentageLoss,percentageLoss
0,India,1979.0,7.2973,8.666699,10.232100,1.5683,27.764399,67.435845,29.559251,5.017527,18.071800,18.071800
1,India,1980.0,7.3077,9.375000,9.961400,1.4356,28.079700,68.533794,30.701423,5.022887,17.887965,17.887965
2,India,1981.0,7.3333,9.473700,10.561999,1.6299,28.998899,70.497130,31.526624,5.228124,18.028698,18.028698
3,India,1982.0,7.7000,9.722199,10.330400,1.6913,29.443898,72.825924,32.220937,5.231355,17.767196,17.767196
4,India,1983.0,7.7073,10.000000,9.981900,1.8158,29.505000,74.281738,32.826625,5.169826,17.521864,17.521864


In [29]:
#fullData.to_csv(processed_data_path + "/finalData2use.csv", index=False)